In [1]:
cd Desktop/

C:\Users\younggy\Desktop


In [2]:
train = pd.read_csv("hw3_train.dat",sep=" ",header=None,names=["x","y","label"])
test = pd.read_csv("hw3_test.dat",sep=" ",header=None,names=["x","y","label"])
print train.shape
print test.shape

(100, 3)
(1000, 3)


In [3]:
#构建node基本结构
class Node():
    def __init__(self,value,parent):
        self.value = value
        self.X = value.iloc[:,0:2]
        self.Y = value.iloc[:,2]
        self.parent = parent
        self.leftchild = None
        self.rightchild = None
        self.axis = None
        self.theta = None
    def getValue(self):
        return self.value
    def getParent(self):
        return self.parent
    
    #下面开始挑选axis和theta
    def gini(self,tr):
        fr = 1 - tr
        return 1- tr**2 - fr**2
    def oneGetThetaImpurity(self,x,y):
        #x,y是panda.dataframe的数据
        #assert type(x)==pd.DataFrame and type(y)==pd.DataFrame
        #输入包含xy的数据d，得到划分的theta值和划分后的impurity
        #需要注意到，如果数据都相等的话，那么需要特别标注，也就是达到了节点
#         if np.mean(np.array(y,dtype="double"))== y[0]:
#             return (None,0)   
        d = pd.concat([x,y],axis=1)
        d.columns = ["x","label"]
        d_sort = d.sort_values(by="x")
        #寻找划分的splittheta
        splittheta = []
        d_sortx = np.array(d_sort["x"])
        for i in range(len(d_sortx)-1):
            splittheta.append((d_sortx[i]+d_sortx[i+1])/2)
        #寻找最佳的splittheta
        bestTheta = None
        bestImpurity = float("inf")
        for theta in splittheta:
            part1 = d_sort.iloc[d_sortx<theta,]
            part2 = d_sort.iloc[d_sortx>=theta,]
            impurity1Dir1 = len(part1)*self.gini(np.mean(part1["label"]==1)) + len(part2)*self.gini(np.mean(part2["label"]==-1))     
            impurity1Dir1 = len(part1)*self.gini(np.mean(part1["label"]==-1)) + len(part2)*self.gini(np.mean(part2["label"]==1)) 
            impurity = min(impurity1Dir1,impurity1Dir1)
            if impurity<bestImpurity:
                bestImpurity = impurity
                bestTheta = theta
        #返回值
        return (bestTheta,bestImpurity)
    
    def getAxisTheta(self):
        #首先判断Y是不是都相等，如果都相等，那么就分不了
        if np.mean(np.array(self.Y,dtype="double"))== self.Y.iloc[0]:
            return None
        #否则继续往下面分
        axis0 = self.X.iloc[:,0]
        axis1 = self.X.iloc[:,1]
        (bestTheta0,bestImpurity0) = self.oneGetThetaImpurity(axis0,self.Y)
        (bestTheta1,bestImpurity1) = self.oneGetThetaImpurity(axis1,self.Y)
        #寻找分类的最佳theta和impurity
        bestTheta = None
        bestAxis = None
        bestImpurity = float("inf")
        if bestImpurity0<bestImpurity1:
            bestAxis = 0
            bestTheta = bestTheta0
            bestImpurity = bestImpurity0
        else:
            bestAxis = 1
            bestTheta = bestTheta1
            bestImpurity = bestImpurity1
        #寻找分类后的最佳part1和part2
        index = np.array(self.X.iloc[:,bestAxis] < bestTheta)
        left = self.value.iloc[index,:]
        right = self.value.iloc[np.logical_not(index),:]
        return (bestAxis,bestTheta,left,right)
           
    def getChilds(self):
        if self.getAxisTheta()==None:
            return None
        (bestAxis,bestTheta,left,right) = self.getAxisTheta()     
        self.leftchild = Node(left,self)
        self.rightchild = Node(right,self)
        self.axis = bestAxis
        self.theta = bestTheta
        return (self.leftchild,self.rightchild)

In [4]:
#构建基本树结构
class Tree():
    def __init__(self):
        self.nodes = []
        self.leaves = []
        self.struct = {}
    def buildTree(self,root):
        if root.getChilds() != None:     
            (leftchild, rightchild)  = root.getChilds()
            self.nodes.append(root)
            self.struct[root] = [leftchild,rightchild]
            self.buildTree(leftchild)
            self.buildTree(rightchild)
        else:
            self.leaves.append(root)
    def predict(self,node,x):
        #确定x的样式
        #x是np.array样式的数组,只能预测单个值
        #定义终止条件
        if node.leftchild==None:
            return node.Y.iloc[0]
        #否则的话不断迭代到终止条件的node
        leftchild = node.leftchild
        rightchild = node.rightchild
        axis = node.axis
        theta = node.theta
        if x[axis]<theta:
            return self.predict(leftchild,x)
        else:
            return self.predict(rightchild,x)
    def predictD(self,node,D):
        return [self.predict(node,i) for i in D]

In [5]:
#测试
d = train
n1 = Node(d,None)
t1 = Tree()
t1.buildTree(n1)
#预测
#np.mean(t1.predictD(n1,np.array(test)[:,0:2]) == test.label)

In [12]:
%%px 
import os
print  os.getpid()

[stdout:0] 2256
[stdout:1] 12324
[stdout:2] 14428
[stdout:3] 11212
[stdout:4] 13268
[stdout:5] 1216
[stdout:6] 15332
[stdout:7] 11356


In [16]:
#取样，并根据取样建立DT
EintrainRF = []
EintestRF = []
EintrainG = []
#100次试验
for i in range(20):
    #每个RF有300个小树
    rf_ptrain = []
    rf_ptest = []
    for j in range(100): #300 trees
        sample = train.sample(len(train),replace=True)
        n1 = Node(sample,None)
        t1 = Tree()
        t1.buildTree(n1)
        #Ein.append(np.mean(t1.predictD(n1,np.array(train)[:,0:2]) != train.label))
        #将RF中每个小树的预测结果储存到大树中
        p_train = t1.predictD(n1,np.array(train)[:,0:2])
        p_test = t1.predictD(n1,np.array(test)[:,0:2])
        rf_ptrain.append(p_train)
        rf_ptest.append(p_test)
        EintrainG.append(np.mean(p_train!=np.array(train["label"])))
    #计算当前大树的正确率
    print "%s big random forest finished" % (i)
    rf_ptrain = np.array(rf_ptrain)
    rf_ptest = np.array(rf_ptest)
    #大树在训练集和测试集上不准确的概率
    EintrainRF.append(np.mean((rf_ptrain.mean(0) >0) !=  (np.array(train["label"])==1)))
    EintestRF.append(np.mean((rf_ptest.mean(0) >0) !=  (np.array(test["label"])==1)))

0 big random forest finished
1 big random forest finished
2 big random forest finished
3 big random forest finished
4 big random forest finished
5 big random forest finished
6 big random forest finished
7 big random forest finished
8 big random forest finished
9 big random forest finished
10 big random forest finished
11 big random forest finished
12 big random forest finished
13 big random forest finished
14 big random forest finished
15 big random forest finished
16 big random forest finished
17 big random forest finished
18 big random forest finished
19 big random forest finished


C:\Users\younggy\Anaconda\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
